# Interface Segregation Principle (ISP)

> “Clients should not be forced to depend on interfaces they do not use.” – Robert C. Martin

## 1. Concept
* Each **interface** is a *contract oriented to a specific client need*.
* When a class implements an interface it should use **all** its members; if it ignores half of them, the interface is *too fat*.
* ISP complements **Liskov Substitution Principle**: fat interfaces often push sub‑classes to violate contracts (throwing, returning `null`). By splitting contracts, we preserve substitutability.

### Relationship with LSP & OCP
* **LSP** demands that every subtype fulfils its parent’s contract. Fat interfaces make that hard; ISP trims contracts so sub‑types can honour them.
* Smaller, cohesive interfaces also help **OCP** because new behaviour can be added via new interfaces instead of modifying old ones.

## 2. Problematic design without ISP

In [1]:
// BAD: Fat interface - simple devices must implement unused members
interface MultifunctionDevice {
  print(doc: string): void;
  scan(doc: string): void;
  fax(doc: string): void;
}

class OldFashionedPrinter implements MultifunctionDevice {
  print(doc: string): void {
    console.log(`Printing: ${doc}`);
  }
  scan(doc: string): void {
    // Device has no scanner → LSP violation
    throw new Error("Scan not supported");
  }
  fax(doc: string): void {
    // Device has no fax → LSP violation
    throw new Error("Fax not supported");
  }
}

// Client interested only in printing
function printDocument(device: MultifunctionDevice, doc: string) {
  device.print(doc);
}

const legacy = new OldFashionedPrinter();
printDocument(legacy, "report.pdf");   // works but device still contains dead methods

Printing: report.pdf


### Issues
1. **Unused methods** → wasted implementation, risk of runtime exceptions.
2. **Violates LSP** – sub‑classes throw errors, tightening pre‑conditions.
3. Leads to brittle code; changing interface affects many unrelated classes.

## 3. Refactor applying ISP – split contracts

In [2]:
// GOOD: Separate smaller, client‑specific interfaces
interface Printer {
  print(doc: string): void;
}

interface Scanner {
  scan(doc: string): void;
}

interface Fax {
  fax(doc: string): void;
}

// Concrete devices pick only the abilities they truly support
class BasicPrinter implements Printer {
  print(doc: string) {
    console.log(`Printing: ${doc}`);
  }
}

class AllInOne implements Printer, Scanner, Fax {
  print(doc: string) { console.log(`AIO printing ${doc}`); }
  scan(doc: string) { console.log(`AIO scanning ${doc}`); }
  fax(doc: string) { console.log(`AIO faxing ${doc}`); }
}

// Client code depends only on the relevant capability
function printDocs(device: Printer, docs: string[]) {
  docs.forEach(d => device.print(d));
}

const homePrinter = new BasicPrinter();
const officeAIO = new AllInOne();

printDocs(homePrinter, ["cv.pdf"]);
printDocs(officeAIO, ["contract.pdf"]);

Printing: cv.pdf
AIO printing contract.pdf


### Benefits achieved
* **Cohesion** – each interface models a single capability.
* **Substitution safe** – no method ever throws `NotImplemented`.
* **Flexibility** – mix capabilities through composition instead of monolithic inheritance.

## 4. Trade‑offs & Constraints
* Potentially **more interfaces** to manage.
* Over‑segmentation can clutter autocompletion for developers if granularity is excessive.
* Requires thoughtful domain analysis to find the right boundaries.

## 5. Spotting ISP violations – quick checklist
1. Classes implement interfaces but leave methods empty or throw `Error`.
2. Interface changes cause *many* unrelated classes to recompile.
3. Clients pass `null` for parameters in a method they don't need.
4. Presence of comments like `// not used by X` next to interface members.

## 6. Related GoF patterns
* **Facade** – group multiple small interfaces under a simplified facade for convenience.
* **Adapter** – adapt a wide interface to a narrow one expected by the client.
* **Decorator** – add behaviour without bloating the core interface.

## 7. References
* Robert C. Martin, *Agile Software Development: Principles, Patterns, and Practices* (2002)
* Barbara Liskov, *A Behavioral Notion of Subtyping* (1994)
* https://principles.dev/#isp